In [58]:
%%time
import warnings
warnings.filterwarnings("ignore")
import pyart
import pydda
import netCDF4 as nc
import xarray as xr
import datetime as dt
import numpy as np
import wradlib as wrl
from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter, FuncFormatter
import pandas as pd
from matplotlib import axes
from matplotlib.ticker import NullFormatter
import cartopy.crs as ccrs
import cartopy.feature as feat
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import glob, os, sys
import shutil
import pathlib
%matplotlib inline

CPU times: user 1.46 ms, sys: 61 µs, total: 1.52 ms
Wall time: 1.79 ms


In [23]:
def add_map(ax, b = 0, l = 0):
    gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.3, color='black', alpha=0.3, 
                         linestyle='-', draw_labels=True)
    gl.xlabels_top = False
    gl.xlabels_bottom = b
    gl.ylabels_left = l
    gl.ylabels_right= False
    gl.xlines = True
    gl.ylines = True
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    ax.add_feature(feat.BORDERS, lw = 0.5)
    ax.add_feature(feat.LAND, lw = 0.3, fc = [0.9,0.9,0.9])
    ax.add_feature(feat.COASTLINE, lw = 0.5)
    ax.add_feature(feat.OCEAN, alpha = 0.5)
    ax.add_feature(feat.STATES.with_scale("10m"), alpha = 0.5, lw = 0.5)

In [24]:
def test_radar(file=None, moment="DBZHC_F"):
    myradar = pyart.io.read(file)
    myradar.scan_type = "ppi"
    mydisplay = pyart.graph.RadarMapDisplay(myradar)
    if moment.lower().startswith("d") or  moment.lower().startswith("r"):
        cmap = "pyart_NWSRef"
    elif moment.lower().startswith("v"):
        cmap = "pyart_NWSVel"
    else:
        cmap = "turbo"
    mydisplay.plot(moment, cmap=cmap)
    return myradar

In [54]:
files = sorted(glob.glob("COW1/*_v1_*SUR.nc"))
len(files)

156

In [55]:
sweeps = []
temp = []
for i in range(len(files)-1):
    s_pos = int(files[i].find('s'))
    st_pos = files[i][s_pos+1:s_pos+3]
    s_pos1 = int(files[i+1].find('s'))
    end_pos = files[i+1][s_pos1+1:s_pos1+3]
#     print(files[i][s_pos+1:s_pos+3],files[i+1][s_pos1+1:s_pos1+3])
    temp.append(files[i])
    print("len(files):",len(files),i)
    print(st_pos,end_pos)
#     print()
    if st_pos>end_pos:
        print("temp len: ", len(temp))
        sweeps.append(temp)
        temp = []
    elif len(files)-i==2:
        print('entered')
        sweeps.append(temp)

len(files): 156 0
01 02
len(files): 156 1
02 03
len(files): 156 2
03 04
len(files): 156 3
04 05
len(files): 156 4
05 06
len(files): 156 5
06 07
len(files): 156 6
07 08
len(files): 156 7
08 09
len(files): 156 8
09 10
len(files): 156 9
10 11
len(files): 156 10
11 12
len(files): 156 11
12 13
len(files): 156 12
13 14
len(files): 156 13
14 15
len(files): 156 14
15 16
len(files): 156 15
16 17
len(files): 156 16
17 18
len(files): 156 17
18 19
len(files): 156 18
19 20
len(files): 156 19
20 21
len(files): 156 20
21 22
len(files): 156 21
22 23
len(files): 156 22
23 24
len(files): 156 23
24 25
len(files): 156 24
25 26
len(files): 156 25
26 27
len(files): 156 26
27 28
len(files): 156 27
28 29
len(files): 156 28
29 30
len(files): 156 29
30 31
len(files): 156 30
31 32
len(files): 156 31
32 33
len(files): 156 32
33 34
len(files): 156 33
34 35
len(files): 156 34
35 36
len(files): 156 35
36 37
len(files): 156 36
37 38
len(files): 156 37
38 39
len(files): 156 38
39 01
temp len:  39
len(files): 156 39
01

In [57]:
sweeps

[['COW1/cfrad.20220322_171015.498_COW1_v1_s01_el0.50_SUR.nc',
  'COW1/cfrad.20220322_171028.423_COW1_v1_s02_el1.20_SUR.nc',
  'COW1/cfrad.20220322_171041.296_COW1_v1_s03_el1.90_SUR.nc',
  'COW1/cfrad.20220322_171054.162_COW1_v1_s04_el2.61_SUR.nc',
  'COW1/cfrad.20220322_171107.031_COW1_v1_s05_el3.40_SUR.nc',
  'COW1/cfrad.20220322_171119.909_COW1_v1_s06_el4.22_SUR.nc',
  'COW1/cfrad.20220322_171132.775_COW1_v1_s07_el5.09_SUR.nc',
  'COW1/cfrad.20220322_171145.652_COW1_v1_s08_el0.48_SUR.nc',
  'COW1/cfrad.20220322_171158.518_COW1_v1_s09_el1.20_SUR.nc',
  'COW1/cfrad.20220322_171211.389_COW1_v1_s10_el1.90_SUR.nc',
  'COW1/cfrad.20220322_171224.269_COW1_v1_s11_el2.61_SUR.nc',
  'COW1/cfrad.20220322_171237.132_COW1_v1_s12_el3.40_SUR.nc',
  'COW1/cfrad.20220322_171250.010_COW1_v1_s13_el4.22_SUR.nc',
  'COW1/cfrad.20220322_171302.883_COW1_v1_s14_el8.01_SUR.nc',
  'COW1/cfrad.20220322_171315.755_COW1_v1_s15_el11.02_SUR.nc',
  'COW1/cfrad.20220322_171328.629_COW1_v1_s16_el14.03_SUR.nc',
  'COW

In [15]:
dir_names = []
for n in range(len(sweeps)):
    dir_names.append('temp_sweeps_{}'.format(n))

In [56]:
for num in range(len(sweeps)):
    print("agg_output/"+dir_names[num])
    for file in sweeps[num]:
#         print(file)
        pathlib.Path("agg_output/"+dir_names[num]).mkdir(parents=True, exist_ok=True)
        shutil.move(file, "agg_output/"+dir_names[num])

agg_output/temp_sweeps_0
agg_output/temp_sweeps_1
agg_output/temp_sweeps_2
agg_output/temp_sweeps_3


In [74]:
for num in range(len(sweeps)):
    new_files = (sorted(glob.glob(os.path.join("agg_output/",dir_names[num]))))
    for file in new_files:
        print(file)
        !RadxConvert -ag_all -f $file/* -debug -outdir ./

agg_output/temp_sweeps_0
Program 'RadxConvert'
Run-time 2022/11/25 08:40:14.

Copyright (c) 1992 - 2022
University Corporation for Atmospheric Research (UCAR)
National Center for Atmospheric Research (NCAR)
Boulder, Colorado, USA.

Redistribution and use in source and binary forms, with
or without modification, are permitted provided that the following
conditions are met:

1) Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

2) Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

3) Neither the name of UCAR, NCAR nor the names of its contributors, if
any, may be used to endorse or promote products derived from this
software without specific prior written permission.

4) If the software is modified to produce derivative works, such modified
software should be clea

INFO: RadxFile::writeToDir
  Writing CfRadial file to dir: ./
DEBUG - NcfRadxFile::writeToDir
  Writing to dir: ./
DEBUG - NcfRadxFile::writeToPath
  Writing to path: .//20220322/cfrad.20220322_184025.476_to_20220322_184835.528_COW1high_SUR.nc
  Tmp path is: .//20220322/tmp.8486.1669365780.499887.tmp
  Writing fields and compressing ...
  ... writing field: NCP
  ... writing field: TRIP_FLA
  ... writing field: SNRHC
  ... writing field: SNRVC
  ... writing field: DBMHC
  ... writing field: DBMVC
  ... writing field: DBZHC
  ... writing field: DBZHC_F
  ... writing field: DBZVC
  ... writing field: DBZVC_F
  ... writing field: VEL
  ... writing field: VEL_F
  ... writing field: VS
  ... writing field: VS_F
  ... writing field: VL
  ... writing field: VL_F
  ... writing field: WIDTH
  ... writing field: ZDRM
  ... writing field: RHOHV
  ... writing field: PHIDP
  ... writing field: KDP
  ... writing field: DBZHCC
  ... writing field: DBZHCC_F
  ... writing field: ZDRC
DEBUG - NcfRadxFil